# Problem
Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/dangkhoa/spark-2.3.1-bin-hadoop2.7')

## Session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Linear_Regression').getOrCreate()

## Load dataset

In [3]:
raw_data = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

raw_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
raw_data.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



## String Indexer

In [5]:
# String Indexer
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(
    inputCol="Cruise_line",
    outputCol="Cruise_line_Index")

string_indexed_data = indexer.fit(raw_data).transform(raw_data)
string_indexed_data.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_Index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
+-----------+-----------+---+------------------+----------+------+------+-------

## Data Exploratory

In [6]:
string_indexed_data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [7]:
from pyspark.sql.functions import corr

string_indexed_data.select(
    corr('crew','Cruise_line_Index'),
    corr('crew','Age'),
    corr('crew','Tonnage'),
    corr('crew','passengers')).show()

+-----------------------------+-------------------+-------------------+----------------------+
|corr(crew, Cruise_line_Index)|    corr(crew, Age)|corr(crew, Tonnage)|corr(crew, passengers)|
+-----------------------------+-------------------+-------------------+----------------------+
|         -0.48332562728617057|-0.5306565039638852|  0.927568811544939|    0.9152341306065384|
+-----------------------------+-------------------+-------------------+----------------------+



In [8]:
string_indexed_data.select(
    corr('crew','length'),
    corr('crew','passenger_density'),
    corr('crew','cabins')).show()

+------------------+-----------------------------+------------------+
|corr(crew, length)|corr(crew, passenger_density)|corr(crew, cabins)|
+------------------+-----------------------------+------------------+
|0.8958566271016579|         -0.15550928421699717|0.9508226063578497|
+------------------+-----------------------------+------------------+



## Vector Assembler
- Grab all columns -> 1 single vector

In [9]:
from pyspark.ml.feature import VectorAssembler

# Define assembler 
assembler = VectorAssembler(
    inputCols=['Tonnage',
               'passengers',
               'length',
               'cabins'],
    outputCol='features')

# transform
vector_indexed_data = assembler.transform(string_indexed_data)
vector_indexed_data \
    .select([
        'Tonnage', 'passengers', 'length', 'cabins',
        'features',
        'crew']) \
    .show(5)

+------------------+----------+------+------+--------------------+----+
|           Tonnage|passengers|length|cabins|            features|crew|
+------------------+----------+------+------+--------------------+----+
|30.276999999999997|      6.94|  5.94|  3.55|[30.2769999999999...|3.55|
|30.276999999999997|      6.94|  5.94|  3.55|[30.2769999999999...|3.55|
|            47.262|     14.86|  7.22|  7.43|[47.262,14.86,7.2...| 6.7|
|             110.0|     29.74|  9.53| 14.88|[110.0,29.74,9.53...|19.1|
|           101.353|     26.42|  8.92| 13.21|[101.353,26.42,8....|10.0|
+------------------+----------+------+------+--------------------+----+
only showing top 5 rows



## Create dataset - Train/Test set

In [10]:
# X = features, y = crew
dataset = vector_indexed_data.select('features', 'crew')

dataset.show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[30.2769999999999...|3.55|
|[30.2769999999999...|3.55|
|[47.262,14.86,7.2...| 6.7|
|[110.0,29.74,9.53...|19.1|
|[101.353,26.42,8....|10.0|
+--------------------+----+
only showing top 5 rows



In [11]:
train_data,test_data = dataset.randomSplit([0.7,0.3])

## Linear Regression

In [12]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    featuresCol='features',
    labelCol='crew',
    predictionCol='prediction')

In [13]:
# Train
model = lr.fit(train_data)

## Model Summary

In [14]:
print("Coefficients: {}\nIntercept: {}".format(model.coefficients,model.intercept))

Coefficients: [0.01106238038784619,-0.1122675787110843,0.4303942062516852,0.7455807066020125]
Intercept: -0.9781181495770807


In [15]:
trainingSummary = model.summary

# train RMSE, MSE
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("MSE: {}".format(trainingSummary.meanSquaredError))
print("R2: {}".format(trainingSummary.r2))

RMSE: 1.073642564205559
MSE: 1.1527083556738875
R2: 0.8996652491789541


## Evaluate Testset

In [16]:
test_results = model.evaluate(test_data)

# test RMSE, MSE
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 0.6590121538629906
MSE: 0.43429701893913797
R2: 0.9685480407306172


## Prediction on Testset

In [17]:
X = test_data.select('features')

predictions = model.transform(X)

In [18]:
# Predict
predictions.show(5)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[3.341,0.66,2.79,...|0.4315861299702636|
|[10.0,2.08,4.4,1.04]|1.5681275329558337|
|[10.0,2.08,4.4,1.04]|1.5681275329558337|
|[14.745,3.08,6.17...| 2.657850661683609|
|[16.8,2.96,5.14,1...|2.1911034738585657|
+--------------------+------------------+
only showing top 5 rows



In [19]:
# Real label
test_data.show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[3.341,0.66,2.79,...|0.59|
|[10.0,2.08,4.4,1.04]| 1.6|
|[10.0,2.08,4.4,1.04]| 1.6|
|[14.745,3.08,6.17...| 1.8|
|[16.8,2.96,5.14,1...|1.97|
+--------------------+----+
only showing top 5 rows

